In [6]:
import pandas as pd
import ast
import re
from itertools import zip_longest

df = pd.read_excel('original_mistral_responses.xlsx')

correct = 0
total = 0
num_word_mismatch = 0

def clean_string(s):
    # Replace double double-quotes with single double-quote
    s = s.replace('""', '"')
    # Replace curly quotes with straight quotes
    s = s.replace('“', '"').replace('”', '"').replace("‘", "'").replace("’", "'")
    # Remove any leading/trailing whitespace
    s = s.strip()
    return s

def safe_eval(s):
    """Try to eval a possibly incomplete list of tuples."""
    try:
        return ast.literal_eval(s)
    except:
        # Try to repair: ensure starts with '[' and ends with ']'
        if not s.startswith("["):
            s = "[" + s
        if not s.endswith("]"):
            s = s.rstrip(", ") + "]"
        try:
            return ast.literal_eval(s)
        except:
            # As last fallback: extract tuples with regex
            tuples = re.findall(r"\('([^']+)',\s*'([^']+)'\)", s)
            return [(w, t) for w, t in tuples]

for idx, row in df.iterrows():
    gt_str = clean_string(row['ground_truth'])
    out_str = clean_string(row['original_response_mistral'])  # adjust column name

    ground_truth = safe_eval(gt_str)
    gpt_answer = safe_eval(out_str)

    for gt, gpt in zip_longest(ground_truth, gpt_answer, fillvalue=None):
        if gt is None or gpt is None:
            continue  
        try:
            (or_word, or_tag), (gpt_word, gpt_tag) = gt, gpt
        except Exception as e:
            print('here')
        if or_word == gpt_word:
            if or_tag == gpt_tag:
                correct += 1
        else:
            num_word_mismatch += 1
            continue
        
        total += 1

accuracy = 100*(correct / total) if total > 0 else 0
print(f"Accuracy: {accuracy:.4f}%")
# print(f"Number of Word Mismatch: {num_word_mismatch} out of Total {total} words")

here
Accuracy: 84.9898%
